## Q1. LOAD YOUR DATA INTO THE TOOL OF YOUR CHOICE AND TAKE A SCREENSHOT

In [1]:
# import necessary libraries and specify that graphs should be plotted inline. 
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from IPython.display import display, HTML
from sklearn import preprocessing
import json
from pprint import pprint
#import plotly as py
#from plotly.tools import FigureFactory as FF
#from ggplot import *
#import plotly.graph_objs as go

In [2]:
import os;
os.chdir('D:/Code/Datasets')

## 1.1 Load the checkin data 

In [12]:
# Let us import the checking data 
d = list()
with open('yelp_academic_dataset_checkin.json','r') as json_data:
    for line in json_data:
        d.append(json.loads(line))
checkin = DataFrame(d)
del d,json_data,line

In [13]:
checkin['tot_checkins'] = checkin['checkin_info'].apply(lambda a : sum(a.values()))

In [14]:
checkin.head(5)

,business_id,checkin_info,type,tot_checkins
0,cE27W9VPgO88Qxe4ol6y_g,"{u'9-5': 1, u'7-5': 1, u'13-3': 1, u'17-6': 1,...",checkin,9
1,mVHrayjG3uZ_RLHkLj-AMg,"{u'11-5': 1, u'9-5': 2, u'9-4': 3, u'15-1': 1,...",checkin,28
2,mYSpR_SLPgUVymYOvTQd_Q,"{u'5-5': 1, u'5-4': 1, u'9-3': 1, u'11-4': 1, ...",checkin,8
3,KayYbHCt-RkbGcPdGOThNg,"{u'23-1': 1, u'11-2': 2, u'21-4': 1, u'16-1': ...",checkin,65
4,b12U9TFESStdy7CsTtcOeg,"{u'5-5': 2, u'13-2': 1, u'8-6': 1, u'5-1': 2, ...",checkin,7


## 1.2 Load the business data 

In [3]:
# Let us import the checking data 

d = list()
with open('yelp_academic_dataset_business.json','r') as json_data:
    for line in json_data:
        d.append(json.loads(line))
business = DataFrame(d)
del d,json_data,line

In [8]:
def find_rest(a_list,x):
    res = [y for y in a_list if x in y]
    if len(res) >= 1:
        return True
    else:
        return False

### Filtering restaurant data into a new dataframe

In [9]:
restaurants = business[business['categories'].apply(lambda a : find_rest(a,'Restaurants'))].reset_index()
restaurants = restaurants.drop('index',axis=1)

In [10]:
restaurants.head(2)

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,"{u'Take-out': True, u'Drive-Thru': False, u'Ou...",5UmKMjUEUNdYWqANhGckJw,"[Fast Food, Restaurants]",Dravosburg,"4734 Lebanon Church Rd\nDravosburg, PA 15034","{u'Tuesday': {u'close': u'21:00', u'open': u'1...",40.354327,-79.900706,Mr Hoagie,[],True,7,3.5,PA,business
1,"{u'Alcohol': u'full_bar', u'Noise Level': u'av...",mVHrayjG3uZ_RLHkLj-AMg,"[Bars, American (New), Nightlife, Lounges, Res...",Braddock,"414 Hawkins Ave\nBraddock, PA 15104","{u'Tuesday': {u'close': u'19:00', u'open': u'1...",40.408830,-79.866211,Emil's Lounge,[],True,26,4.5,PA,business


In [13]:
def get_restaurant_attr(a_json,column):
    if column == 'Ambience':
        if column in a_json.keys(): 
            return sum(a_json[column].values())
    if column == 'Good For':
        if column in a_json.keys(): 
            if sum(a_json[column].values()) > 0 : return 1
            else : return 0
    if column == 'Parking':
        if column in a_json.keys(): 
            return sum(a_json[column].values())
    elif column in a_json.keys():
        return a_json[column]
    else:
        return 'N/A'

In [17]:
restaurants['Acc_credit_card'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Accepts Credit Cards'))
restaurants['Alcohol'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Alcohol'))
restaurants['Ambience'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Ambience'))
restaurants['Delivery'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Delivery'))
restaurants['Good For'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Good For'))
restaurants['Parking'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Parking'))
restaurants['Price Range'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Price Range'))
restaurants['Take-out'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Take-out'))
restaurants['Takes Reservations'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Takes Reservations'))
restaurants['Waiter Service'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Waiter Service'))
restaurants['Noise Level'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Noise Level'))
restaurants['Outdoor Seating'] = restaurants['attributes'].apply(lambda x : get_restaurant_attr(x,'Outdoor Seating'))

In [18]:
restaurants_charlotte = restaurants[restaurants['city']=='Charlotte']

In [19]:
del business

## 1.3 Load the Review data 

In [20]:
d = list()
with open('yelp_academic_dataset_review.json','r') as json_data:
    for line in json_data:
        d.append(json.loads(line))
review = DataFrame(d)
del d,json_data,line

In [21]:
review['useful'] = review['votes'].apply(lambda a : a['useful'])
review['funny'] = review['votes'].apply(lambda a : a['funny'])
review['cool'] = review['votes'].apply(lambda a : a['cool'])

In [22]:
review.head(2)

,business_id,date,review_id,stars,text,type,user_id,votes,useful,funny,cool
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,Ya85v4eqdd6k9Od8HbQjyA,4,"Mr Hoagie is an institution. Walking in, it do...",review,PUFPaY9KxDAcGqfsorJp3Q,"{u'funny': 0, u'useful': 0, u'cool': 0}",0,0,0
1,5UmKMjUEUNdYWqANhGckJw,2014-02-13,KPvLNJ21_4wbYNctrOwWdQ,5,Excellent food. Superb customer service. I mis...,review,Iu6AxdBYGR4A0wspR9BYHA,"{u'funny': 0, u'useful': 0, u'cool': 0}",0,0,0


In [23]:
restaurants_review = pd.merge(review,restaurants_charlotte[['business_id','name']], on='business_id')

In [24]:
restaurants_review

,business_id,date,review_id,stars,text,type,user_id,votes,useful,funny,cool,name
0,Gw5POFxBRoJxN_ZYhk6lgQ,2010-12-31,5jK4kzMbk5_2uFlW6SBqdQ,4,In the Pineville area and need a quick breakfa...,review,whKpnugMnWTz3UjlrN71yg,"{u'funny': 1, u'useful': 1, u'cool': 1}",1,1,1,Bruegger's Bagels
1,Gw5POFxBRoJxN_ZYhk6lgQ,2011-07-18,GLBryAnx1thVohzWc1nV_A,5,I came here for the first time and was so happ...,review,4522K1jh68XVpUsPh-5I7A,"{u'funny': 0, u'useful': 0, u'cool': 0}",0,0,0,Bruegger's Bagels
2,Gw5POFxBRoJxN_ZYhk6lgQ,2012-09-19,ZXPMc6Lc4V7nqknsJ13u9A,4,This place is an awesome!\n\nThe bagels are fr...,review,PM2jXrlVzik1jDwwahLJJQ,"{u'funny': 0, u'useful': 0, u'cool': 0}",0,0,0,Bruegger's Bagels
3,Gw5POFxBRoJxN_ZYhk6lgQ,2012-12-29,DttWnjejrfXUDiZzC0aFAg,3,I have mixed feelings about Bruegger's but I s...,review,naBs2Kg2KClrKcoRgFE2eA,"{u'funny': 0, u'useful': 0, u'cool': 0}",0,0,0,Bruegger's Bagels
4,Gw5POFxBRoJxN_ZYhk6lgQ,2013-06-16,zI76vgTW_MxrqEhl31FuZw,3,Brueggers Bagels when hot and fresh the bagel ...,review,mHGxSFGMYj4uDap-fcAvaw,"{u'funny': 0, u'useful': 0, u'cool': 0}",0,0,0,Bruegger's Bagels
5,Gw5POFxBRoJxN_ZYhk6lgQ,2013-11-10,LNbRXormHaHJv8Cm_7tuiA,5,Great bagles and great ciabatta! We had one of...,review,izn2q7eRSq4XtlJMzoaonA,"{u'funny': 0, u'useful': 0, u'cool': 0}",0,0,0,Bruegger's Bagels
6,Gw5POFxBRoJxN_ZYhk6lgQ,2014-05-25,a5prO8UITURPC_iaQBalgw,1,Cream cheese skimp alert!! We pay ~$7 per smok...,review,sc8OSFfftZk8dvJiKaH1Dg,"{u'funny': 0, u'useful': 0, u'cool': 0}",0,0,0,Bruegger's Bagels
7,Gw5POFxBRoJxN_ZYhk6lgQ,2014-10-08,N_1pNPbWuvGoyU3o0Eg8jg,5,"Admittedly, the grump factor is high in some o...",review,LkfzD47YI70jabh_9ViFdg,"{u'funny': 0, u'useful': 0, u'cool': 0}",0,0,0,Bruegger's Bagels
8,Gw5POFxBRoJxN_ZYhk6lgQ,2015-03-06,kzzKuCzP0xekcBRPiMBTkA,5,Truly one of the BEST bagels I have ever had. ...,review,4FQPzJqtRi94aZWJ9Xo5iA,"{u'funny': 0, u'useful': 0, u'cool': 0}",0,0,0,Bruegger's Bagels
9,Gw5POFxBRoJxN_ZYhk6lgQ,2015-09-18,deDcy6pD4CjOK7ceWx9nGQ,3,I'm all about a good cup of joe with my bagel....,review,-Q00KxShmMTQB50HZCfJ-Q,"{u'funny': 0, u'useful': 0, u'cool': 0}",0,0,0,Bruegger's Bagels


In [25]:
del review

## 1.5 Load the User data 

In [23]:
d = list()
with open('yelp_academic_dataset_user.json','r') as json_data:
    for line in json_data:
        d.append(json.loads(line))
user = DataFrame(d)
del d,json_data,line

In [24]:
user['useful'] = user['votes'].apply(lambda a : a['useful'])
user['funny'] = user['votes'].apply(lambda a : a['funny'])
user['cool'] = user['votes'].apply(lambda a : a['cool'])

In [25]:
user.head(2)

,average_stars,compliments,elite,fans,friends,name,review_count,type,user_id,votes,yelping_since,useful,funny,cool
0,4.14,"{u'profile': 8, u'cute': 15, u'funny': 11, u'p...","[2005, 2006]",69,"[rpOyqD_893cqmDAtJLbdog, 4U9kSBLuBDU391x6bxU-Y...",Russel,108,user,18kPq7GPye-YQ3LyKyAZPw,"{u'funny': 167, u'useful': 282, u'cool': 246}",2004-10,282,167,246
1,3.67,"{u'profile': 117, u'cute': 204, u'funny': 594,...","[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",1345,"[18kPq7GPye-YQ3LyKyAZPw, 4U9kSBLuBDU391x6bxU-Y...",Jeremy,1292,user,rpOyqD_893cqmDAtJLbdog,"{u'funny': 8399, u'useful': 15242, u'cool': 12...",2004-10,15242,8399,12091


## Q2. OUTLINE THE MODELING STEPS YOU ARE GOING TO PERFORM

Modeling steps we will be performing:
<ul>
    <li>1) Colloborative filtering</li>
        <ul><li>For this we will be using the user restaurant reviews to recommed new restaurants to users.</li></ul>
    <br>
    <li>2) Content based recommender system</li>
    <br>
          <p1><i><u>For Content based recommender systems we are going to use the following restaurant features:</u><i></p1>

            <ul><li>review_count: Total number of reviews for that restaurant. </li></ul>
            <ul><li>categories: We want to feature select and use the most informative 20 categories from the list of 306 total categories.</li></ul>
            <ul><li>hours : Average hours the restaurant is open.</li></ul>
            <ul><li>city</li></ul>
            <ul><li>neighborhoods</li></ul>
            <ul><li>tot_checkins</li></ul>
            <ul><li>Qualitative attributes from reviews: We will topic model the reviews from all the users and find out restaurant characterstics which matter the most. We will use 10 words as features for each restaurant and then score them based on the frequency of the words used in the reviews for that restaurant </li></ul>
    <br>
    <li>3) Vectorization and low rank matrix factorization</li>
        <ul><li>We will be using the same features as used in content based recommender systems.</li></ul>
</ul>

## Q3. DESCRIBE BRIEFLY THE USE-CASE SCENARIO OF YOUR MODEL

Recommendation systems are key to the success of any business. We aim to make relevant recommendations that match the user based on the restaurant profile and some information mined from the texual data coming from reviews.
The model built shall be used to provide hotel recommendations to users.

## Q4. WHAT KIND OF EVALUATION ARE YOU PLANNING ON DOING?

### Explanation

We are going to follow K fold cross validation technique. We need to split the test set K times otherwise our results
will have high probability of  being biased

Methodology:
<ul>
    <li>1) We have users, restaurants and the corresponding ratings in a table.</li>
    <li>2) Split  Training and Test set.</li>
    <li>3) Run the recommender system on training set and using cosine similarity, obtain ratings and compare it with the test set using K-fold cross validation.</li>
    <li>4) Measure the error using RMSE by comparing recommended with actual ratings (on test set).</li>
    <li>5) Goal is to minimize RMSE.</li>
</ul>

## 3 Simple Popularity Model

In [30]:
import graphlab as gl
from sklearn.cross_validation import train_test_split

In [26]:
def  normalize_ratings(df):
    mean_rt = DataFrame({'avg_stars' : df.groupby( ['user_id'])['stars'].mean()}).reset_index()
    df = pd.merge(df,mean_rt,on='user_id')
    df['normalized_stars'] = df['stars']- df['avg_stars']
    return df    

In [27]:
pop_data = restaurants_review[['business_id', 'user_id','stars','name']]
pop_data = normalize_ratings(pop_data)
#pop_data = gl.SFrame(pop_data)

In [28]:
pop_data.head()

,business_id,user_id,stars,name,avg_stars,normalized_stars
0,Gw5POFxBRoJxN_ZYhk6lgQ,whKpnugMnWTz3UjlrN71yg,4,Bruegger's Bagels,3.71223,0.28777
1,Lt9o2RmacVY2L-IWLA8uTg,whKpnugMnWTz3UjlrN71yg,5,Morton's The Steakhouse,3.71223,1.28777
2,gpCtDCSXIoQx70cq9itfNA,whKpnugMnWTz3UjlrN71yg,4,Matt's Chicago Dog,3.71223,0.28777
3,-6JY1T0w4XETux5kGOj84g,whKpnugMnWTz3UjlrN71yg,4,Fox and Hound Smokehouse & Tavern,3.71223,0.28777
4,9fxTX1uyyYF4ugx68AG2sg,whKpnugMnWTz3UjlrN71yg,3,Presto Bar & Grill,3.71223,-0.71223


In [31]:
tr, te= train_test_split(pop_data,test_size = 0.3, random_state = 1210)

In [32]:
pop_data_train = gl.SFrame(tr)
pop_data_test = gl.SFrame(te)

This non-commercial license of GraphLab Create for academic use is assigned to kalip004@umn.edu and will expire on November 20, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\surya\AppData\Local\Temp\graphlab_server_1487406006.log.0


In [33]:
pop_data_train.head(5)

business_id,user_id,stars,name,avg_stars,normalized_stars
mVqQ3jT6WvhWd_-vUlPX6Q,XVZCDccLPdYdaj9docxTHg,4,Zaxby's,2.33333333333,1.66666666667
MC8Fv4Hd4reUsTKJYoZPfg,r3SZlDDYGi4yVBFa3ZQZqw,5,JackBeagles,3.94736842105,1.05263157895
awgH-AOkuEX22F9ObdVHIA,ONlOu7pJWk3_55QUYsUFuw,4,Pei Wei,3.77777777778,0.222222222222
wfkuFh39uIaXIDCpkUJV2A,cs3jBn4NMKBjMU7cy1HdRw,1,Emzy Sushi Bar & AsianKitchen ...,3.32142857143,-2.32142857143
Gfke1AcTzc2nlVi1rftrTw,-pqh6Zdnlf1FG_hOBKl3cQ,5,Laurel Market,4.5,0.5


In [34]:
popularity_model = gl.popularity_recommender.create(pop_data_train,\
                                user_id='user_id', item_id='business_id', target='stars')

Recsys training: model = popularity

Warning: Ignoring columns name, avg_stars, normalized_stars;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 70721 observations with 25809 users and 1999 items.

Data prepared in: 0.233088s

70721 observations to process; with 1999 unique items.

In [35]:
#Get recommendations for first 5 users and print them
#users = range(1,6) specifies user ID of first 5 users
#k=5 specifies top 5 recommendations to be given
users_info = list(pop_data['user_id']) 
popularity_recomm = popularity_model.recommend(users=users_info,k=5)
#popularity_recomm.print_rows(num_rows=25)
popularity_recomm = popularity_recomm.to_dataframe()

recommendations finished on 1000/101030 queries. users per second: 112007

recommendations finished on 2000/101030 queries. users per second: 125415

recommendations finished on 3000/101030 queries. users per second: 130628

recommendations finished on 4000/101030 queries. users per second: 94077.8

recommendations finished on 5000/101030 queries. users per second: 98933.5

...

recommendations finished on 97000/101030 queries. users per second: 103147

recommendations finished on 98000/101030 queries. users per second: 102570

recommendations finished on 99000/101030 queries. users per second: 102169

recommendations finished on 100000/101030 queries. users per second: 102249

recommendations finished on 101000/101030 queries. users per second: 100690

In [36]:
popularity_recomm =  pd.merge(popularity_recomm,restaurants[['business_id','name']],on='business_id')

In [38]:
popularity_recomm.head(100)

,user_id,business_id,score,rank,name
0,whKpnugMnWTz3UjlrN71yg,eGdr777YxPCBwDPEWVswSQ,5.0,1,Delicias Restaurante Latino
1,whKpnugMnWTz3UjlrN71yg,eGdr777YxPCBwDPEWVswSQ,5.0,1,Delicias Restaurante Latino
2,whKpnugMnWTz3UjlrN71yg,eGdr777YxPCBwDPEWVswSQ,5.0,1,Delicias Restaurante Latino
3,whKpnugMnWTz3UjlrN71yg,eGdr777YxPCBwDPEWVswSQ,5.0,1,Delicias Restaurante Latino
4,whKpnugMnWTz3UjlrN71yg,eGdr777YxPCBwDPEWVswSQ,5.0,1,Delicias Restaurante Latino
5,whKpnugMnWTz3UjlrN71yg,eGdr777YxPCBwDPEWVswSQ,5.0,1,Delicias Restaurante Latino
6,whKpnugMnWTz3UjlrN71yg,eGdr777YxPCBwDPEWVswSQ,5.0,1,Delicias Restaurante Latino
7,whKpnugMnWTz3UjlrN71yg,eGdr777YxPCBwDPEWVswSQ,5.0,1,Delicias Restaurante Latino
8,whKpnugMnWTz3UjlrN71yg,eGdr777YxPCBwDPEWVswSQ,5.0,1,Delicias Restaurante Latino
9,whKpnugMnWTz3UjlrN71yg,eGdr777YxPCBwDPEWVswSQ,5.0,1,Delicias Restaurante Latino


In [39]:
popularity_model.evaluate(pop_data_test)

recommendations finished on 1000/14839 queries. users per second: 60448.5

recommendations finished on 2000/14839 queries. users per second: 67617.8

recommendations finished on 3000/14839 queries. users per second: 56454.6

recommendations finished on 4000/14839 queries. users per second: 60444.9

recommendations finished on 5000/14839 queries. users per second: 56991.1

...

recommendations finished on 12000/14839 queries. users per second: 40093.7

recommendations finished on 13000/14839 queries. users per second: 39348.9

recommendations finished on 14000/14839 queries. users per second: 39893.5


Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    | 0.000134779971696 | 8.98533144641e-05 |
|   2    | 0.000202169957544 | 0.000184875747161 |
|   3    | 0.000179706628928 | 0.000241034068701 |
|   4    |  0.00016847496462 |  0.00027869317844 |
|   5    | 0.000161735966035 | 0.000379778157212 |
|   6    | 0.000157243300312 | 0.000407425330893 |
|   7    | 0.000163661394203 | 0.000553436966897 |
|   8    | 0.000160051216389 | 0.000637674449207 |
|   9    | 0.000172218852723 | 0.000720227181871 |
|   10   | 0.000175213963205 | 0.000922397139416 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 1.1988239955576159)

Per User RMSE (best)
+------------------------+-------+------+
|        user_id         | count | rmse |
+------------------------+-------+------+
| dkeasvczMuTV4J3-rS3Co

{'precision_recall_by_user': Columns:
 	user_id	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 267102
 
 Data:
 +------------------------+--------+-----------+--------+-------+
 |        user_id         | cutoff | precision | recall | count |
 +------------------------+--------+-----------+--------+-------+
 | --9jRaeY1xK-2l9r9fVQWA |   1    |    0.0    |  0.0   |   1   |
 | --9jRaeY1xK-2l9r9fVQWA |   2    |    0.0    |  0.0   |   1   |
 | --9jRaeY1xK-2l9r9fVQWA |   3    |    0.0    |  0.0   |   1   |
 | --9jRaeY1xK-2l9r9fVQWA |   4    |    0.0    |  0.0   |   1   |
 | --9jRaeY1xK-2l9r9fVQWA |   5    |    0.0    |  0.0   |   1   |
 | --9jRaeY1xK-2l9r9fVQWA |   6    |    0.0    |  0.0   |   1   |
 | --9jRaeY1xK-2l9r9fVQWA |   7    |    0.0    |  0.0   |   1   |
 | --9jRaeY1xK-2l9r9fVQWA |   8    |    0.0    |  0.0   |   1   |
 | --9jRaeY1xK-2l9r9fVQWA |   9    |    0.0    |  0.0   |   1   |
 | --9jRaeY1xK-2l9r9fVQWA |   10   |    0.0    |  0.0   |   1   |
 +-----

## 4 Collaborative Filtering Model item based

In [40]:
#Train Model
item_bases_model = gl.item_similarity_recommender.create(pop_data_train, \
                                        user_id='user_id', item_id='business_id', target='stars', similarity_type='pearson')

Recsys training: model = item_similarity

Warning: Ignoring columns name, avg_stars, normalized_stars;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 70721 observations with 25809 users and 1999 items.

...

| 136.863ms                           | 100              | 1999            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.15854s

In [41]:
#Make Recommendations:
users_info = list(pop_data_train['user_id']) 
item_based_recomm = item_bases_model.recommend(users=users_info,k=5)
item_based_recomm = item_based_recomm.to_dataframe()

recommendations finished on 1000/70721 queries. users per second: 35620.1

recommendations finished on 2000/70721 queries. users per second: 33808.9

recommendations finished on 3000/70721 queries. users per second: 33997.8

recommendations finished on 4000/70721 queries. users per second: 32834.5

recommendations finished on 5000/70721 queries. users per second: 27101.7

...

recommendations finished on 66000/70721 queries. users per second: 34265.2

recommendations finished on 67000/70721 queries. users per second: 34293.5

recommendations finished on 68000/70721 queries. users per second: 34199.8

recommendations finished on 69000/70721 queries. users per second: 34202.5

recommendations finished on 70000/70721 queries. users per second: 33531.6

In [42]:
item_based_recomm.head(5)

,user_id,business_id,score,rank
0,XVZCDccLPdYdaj9docxTHg,eGdr777YxPCBwDPEWVswSQ,5.0,1
1,XVZCDccLPdYdaj9docxTHg,3TolC4iUceA-C52DOJ8G8Q,5.0,2
2,XVZCDccLPdYdaj9docxTHg,X6fm6mRM1vuWLgvEy5VoYA,5.0,3
3,XVZCDccLPdYdaj9docxTHg,mPhpRGTgusNHOXgvF43FQg,5.0,4
4,XVZCDccLPdYdaj9docxTHg,64AT1yFmMwzRM035Z4YduA,5.0,5


## Evaluating Recommendation Engines

In [44]:
model_performance = gl.compare(pop_data_test, [popularity_model, item_bases_model])
gl.show_comparison(model_performance,[popularity_model, item_bases_model])

PROGRESS: Evaluate model M0


recommendations finished on 1000/14839 queries. users per second: 79789.4

recommendations finished on 2000/14839 queries. users per second: 88656.4

recommendations finished on 3000/14839 queries. users per second: 73877.1

recommendations finished on 4000/14839 queries. users per second: 69380.6

...

recommendations finished on 11000/14839 queries. users per second: 20956.5

recommendations finished on 12000/14839 queries. users per second: 20319.4

recommendations finished on 13000/14839 queries. users per second: 20728.1

recommendations finished on 14000/14839 queries. users per second: 21187.8


Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    | 0.000134779971696 | 0.000101084978772 |
|   2    |  0.00016847496462 |  0.00012701991272 |
|   3    | 0.000179706628928 | 0.000185050178311 |
|   4    | 0.000185322461082 | 0.000328253898238 |
|   5    | 0.000202169957544 | 0.000383587517154 |
|   6    | 0.000190938293236 | 0.000518367488851 |
|   7    | 0.000173288535038 | 0.000585757474699 |
|   8    | 0.000160051216389 | 0.000587132780532 |
|   9    | 0.000142267747902 | 0.000587132780532 |
|   10   | 0.000141518970281 | 0.000659706611446 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

Model compare metric: precision_recall
Canvas is accessible via web browser at the URL: http://localhost:52363/index.html
Opening Canvas in default web browser.
